In [ ]:
# 성재님 xgboost에 feature 추가 / 삭제 / RandomizedSearchCV

In [2]:
import numpy as np
import pandas as pd

import os
import random

import warnings

warnings.filterwarnings(action='ignore')

path = '/opt/ml/input/data/'
dat = pd.read_csv(path + '/FE/FE_total.csv')

In [3]:
##### label encodig
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(dat['assessmentItemID'])
dat['assessmentItemID'] = encoder.transform(dat['assessmentItemID'])

encoder.fit(dat['category_st_qcut_5'])
dat['category_st_qcut_5'] = encoder.transform(dat['category_st_qcut_5'])

encoder.fit(dat['testId'])
dat['testId'] = encoder.transform(dat['testId'])

dat

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5
0,0,5354,975,1,2020-03-24 00:17:11,7224,0,6,1,1,0
1,0,5355,975,1,2020-03-24 00:17:14,7225,3,6,1,2,2
2,0,5356,975,1,2020-03-24 00:17:22,7225,8,6,1,3,2
3,0,5357,975,1,2020-03-24 00:17:29,7225,7,6,1,4,2
4,0,5358,975,1,2020-03-24 00:17:36,7225,7,6,1,5,2
...,...,...,...,...,...,...,...,...,...,...,...
2526695,7441,2373,456,0,2020-06-05 06:50:21,438,24,3,71,5,1
2526696,7441,3909,748,1,2020-08-21 01:06:39,8836,0,4,165,1,0
2526697,7441,3910,748,1,2020-08-21 01:06:50,8836,11,4,165,2,2
2526698,7441,3911,748,1,2020-08-21 01:07:36,8836,46,4,165,3,3


In [11]:
def feature_engineering(df):
    
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df.sort_values(by=['userID','Timestamp'], inplace=True)
    
    #유저들의 문제 풀이수, 정답 수, 정답률을 시간순으로 누적해서 계산
    df['user_correct_answer'] = df.groupby('userID')['answerCode'].transform(lambda x: x.cumsum().shift(1))
    df['user_total_answer'] = df.groupby('userID')['answerCode'].cumcount()
    df['user_acc'] = df['user_correct_answer']/df['user_total_answer']

    # testId와 KnowledgeTag의 전체 정답률은 한번에 계산
    # 아래 데이터는 제출용 데이터셋에 대해서도 재사용
    correct_t = df.groupby(['testId'])['answerCode'].agg(['mean', 'sum'])
    correct_t.columns = ["test_mean", 'test_sum']
    correct_k = df.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'sum'])
    correct_k.columns = ["tag_mean", 'tag_sum']

    df = pd.merge(df, correct_t, on=['testId'], how="left")
    df = pd.merge(df, correct_k, on=['KnowledgeTag'], how="left")
    
    return df

In [14]:
dat = feature_engineering(dat)

In [15]:
dat.head()

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5,user_correct_answer,user_total_answer,user_acc,test_mean,test_sum,tag_mean,tag_sum
0,0,5354,975,1,2020-03-24 00:17:11,7224,0,6,1,1,0,NaN,0,NaN,0.952667,1429,0.957333,718
1,0,5355,975,1,2020-03-24 00:17:14,7225,3,6,1,2,2,1.0,1,1.0,0.952667,1429,0.917067,3439
2,0,5356,975,1,2020-03-24 00:17:22,7225,8,6,1,3,2,2.0,2,1.0,0.952667,1429,0.917067,3439
3,0,5357,975,1,2020-03-24 00:17:29,7225,7,6,1,4,2,3.0,3,1.0,0.952667,1429,0.917067,3439
4,0,5358,975,1,2020-03-24 00:17:36,7225,7,6,1,5,2,4.0,4,1.0,0.952667,1429,0.917067,3439


In [16]:
dat = dat.fillna(0)

In [17]:
_train = dat[dat['answerCode'] >= 0]
_test = dat[dat['answerCode'] < 0]

In [18]:
# 유저 마다 가장 마지막 문제를 맞추는 것을 기준으로 평가 하려함
# test 유저에 경우 가장 마지막 문제 전 문제를 맞추는 것을 기준으로 평가함.
user_final_time = _train.groupby('userID')['Timestamp'].max()
_train['train_valid'] = _train.apply(lambda x : -1 if x.Timestamp == user_final_time[x.userID] else x['answerCode'], axis = 1)
_valid = _train[_train['train_valid'] < 0]
_train = _train[_train['train_valid'] >= 0]

In [19]:
# 모델에 적용하기 전 기본적인 데이터 전처리 부분
## 라벨링, 필요없는 칼럼 제거
_train_value = _train['answerCode']
_train.drop(['Timestamp', 'train_valid', 'answerCode'], axis = 1, inplace = True)

_valid_value = _valid['answerCode']
_valid.drop(['Timestamp', 'train_valid', 'answerCode'], axis = 1, inplace = True)

_test.drop(['Timestamp', 'answerCode'], axis = 1, inplace = True)

In [20]:
_train

,userID,assessmentItemID,testId,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5,user_correct_answer,user_total_answer,user_acc,test_mean,test_sum,tag_mean,tag_sum
0,0,5354,975,7224,0,6,1,1,0,0.0,0,0.000000,0.952667,1429,0.957333,718
1,0,5355,975,7225,3,6,1,2,2,1.0,1,1.000000,0.952667,1429,0.917067,3439
2,0,5356,975,7225,8,6,1,3,2,2.0,2,1.000000,0.952667,1429,0.917067,3439
3,0,5357,975,7225,7,6,1,4,2,3.0,3,1.000000,0.952667,1429,0.917067,3439
4,0,5358,975,7225,7,6,1,5,2,4.0,4,1.000000,0.952667,1429,0.917067,3439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526694,7441,2372,456,438,34,3,71,4,3,1.0,3,0.333333,0.666000,999,0.694889,3127
2526695,7441,2373,456,438,24,3,71,5,1,1.0,4,0.250000,0.666000,999,0.694889,3127
2526696,7441,3909,748,8836,0,4,165,1,0,1.0,5,0.200000,0.652500,783,0.698551,2410
2526697,7441,3910,748,8836,11,4,165,2,2,2.0,6,0.333333,0.652500,783,0.698551,2410


In [22]:
_train = _train.drop(columns='assessmentItemID')
_valid = _valid.drop(columns='assessmentItemID')
_test = _test.drop(columns='assessmentItemID')

In [23]:
_train

,userID,testId,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5,user_correct_answer,user_total_answer,user_acc,test_mean,test_sum,tag_mean,tag_sum
0,0,975,7224,0,6,1,1,0,0.0,0,0.000000,0.952667,1429,0.957333,718
1,0,975,7225,3,6,1,2,2,1.0,1,1.000000,0.952667,1429,0.917067,3439
2,0,975,7225,8,6,1,3,2,2.0,2,1.000000,0.952667,1429,0.917067,3439
3,0,975,7225,7,6,1,4,2,3.0,3,1.000000,0.952667,1429,0.917067,3439
4,0,975,7225,7,6,1,5,2,4.0,4,1.000000,0.952667,1429,0.917067,3439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526694,7441,456,438,34,3,71,4,3,1.0,3,0.333333,0.666000,999,0.694889,3127
2526695,7441,456,438,24,3,71,5,1,1.0,4,0.250000,0.666000,999,0.694889,3127
2526696,7441,748,8836,0,4,165,1,0,1.0,5,0.200000,0.652500,783,0.698551,2410
2526697,7441,748,8836,11,4,165,2,2,2.0,6,0.333333,0.652500,783,0.698551,2410


In [29]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
import xgboost as xgb

In [25]:
param_grid = {
    "learning_rate": [0.0001,0.001, 0.01, 0.1, 1],
    "max_depth": range(3,21,3),
    "gamma": [i/10.0 for i in range(0,5)],
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
    "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]
}

In [26]:
scoring = ['recall']

In [37]:
classifier = xgb.XGBClassifier

In [27]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

In [42]:
random_search = RandomizedSearchCV(estimator=classifier(n_estimators=1000), 
                           param_distributions=param_grid, 
                           n_iter=48,
                           scoring=scoring, 
                           refit='recall', 
                           n_jobs=-1, 
                           cv=kfold, 
                           verbose=0)

In [43]:
random_result = random_search.fit(_train, _train_value, verbose=500,early_stopping_rounds=100, eval_metric='auc',eval_set=[(_valid, _valid_value)])

/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from panda

[0]	validation_0-auc:0.63674
[0]	validation_0-auc:0.72567
[0]	validation_0-auc:0.73033
[0]	validation_0-auc:0.63694
[0]	validation_0-auc:0.72858
[0]	validation_0-auc:0.63723
[0]	validation_0-auc:0.65602
[0]	validation_0-auc:0.65702
[500]	validation_0-auc:0.77380
[500]	validation_0-auc:0.77478
[500]	validation_0-auc:0.77514
[500]	validation_0-auc:0.76228
[500]	validation_0-auc:0.76353
[500]	validation_0-auc:0.76350
[500]	validation_0-auc:0.80961
[500]	validation_0-auc:0.80964
[999]	validation_0-auc:0.78165


/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation_0-auc:0.65934
[999]	validation_0-auc:0.78431
[999]	validation_0-auc:0.78580


/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


...

[0]	validation_0-auc:0.76415
[113]	validation_0-auc:0.78061
[150]	validation_0-auc:0.78914


/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation_0-auc:0.75531


/opt/conda/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/conda/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/conda/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	validation_0-auc:0.75600
[113]	validation_0-auc:0.77242
[180]	validation_0-auc:0.78987
[123]	validation_0-auc:0.77297
[750]	validation_0-auc:0.81298
[999]	validation_0-auc:0.81110
[500]	validation_0-auc:0.80231
[500]	validation_0-auc:0.80091
[500]	validation_0-auc:0.80110
[999]	validation_0-auc:0.80764
[999]	validation_0-auc:0.80635
[999]	validation_0-auc:0.80562
[0]	validation_0-auc:0.61214
[166]	validation_0-auc:0.70441


In [44]:
random_result

RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                           verbosity=None),
                   n_iter=48, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.6,
                                                             0.7, 0.8, 0.9],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.0001, 0.001, 0.01,
                                                          0.1, 1],
                                        'max_depth': range(3, 21, 3),
                                        'reg_alpha': [1e-05, 0.01, 0.1, 1, 10,
                                                      100],
                                        'reg_lambda': [1e-05, 0.01, 0.1, 1, 10,
                                                       100]},
                   refit='recall', scoring=['recall'])

In [48]:
print(f'The best score is {random_result.best_score_:.4f}')
print('The best score standard deviation is', round(random_result.cv_results_['std_test_recall'][random_result.best_index_], 4))
print(f'The best hyperparameters are {random_result.best_params_}')

The best score is 0.9547
The best score standard deviation is 0.0002
The best hyperparameters are {'reg_lambda': 0.1, 'reg_alpha': 100, 'max_depth': 3, 'learning_rate': 0.0001, 'gamma': 0.3, 'colsample_bytree': 0.3}


In [55]:
# valid score를 확인해봄.
# 모델 환경상 test score보다 떨어질 수도. (토론 필요)
from sklearn.metrics import accuracy_score, roc_auc_score

_valid_pred = random_search.predict_proba(_valid)[:,1]
print(roc_auc_score(_valid_value, _valid_pred)) # auc
print(accuracy_score(_valid_value, np.where(_valid_pred >= 0.5, 1, 0))) # acc, 정확도

0.7072197121235435
0.6124256444150694


In [57]:
# import matplotlib.pyplot as plt

# feature_importance = random_result.feature_importances_
# sorted_idx = np.argsort(feature_importance)
# fig = plt.figure(figsize=(12, 6))
# plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align="center")
# plt.yticks(range(len(sorted_idx)), np.array(_valid.columns)[sorted_idx])
# plt.title("Feature Importance")

AttributeError: 'RandomizedSearchCV' object has no attribute 'feature_importances_'

In [73]:
# submission 제출하기 위한 코드
_test_pred = random_search.predict_proba(_test)[:,1]
_test['prediction'] = _test_pred
submission = _test['prediction'].reset_index(drop = True).reset_index()
submission.rename(columns = {'index':'id'}, inplace = True)
submission.to_csv('./output/hyper_add_xg_submission.csv', index = False)